In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=48a6b9693438c77a930c6e83ad5f878be87ad30ad3a3acd78aa6408ee932b2f2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from google.colab import files
import pandas as pd

In [ ]:
files.upload()

Saving place_info_corpus - query.csv to place_info_corpus - query.csv


{'place_info_corpus - query.csv': b'place_name,place_info,corpus,"coordinates (y, x)",query,\r\n\xeb\xb2\xa4\xec\xb9\x981,\xea\xb3\xb5\xeb\x8c\x80(\xea\xb3\xb5\xea\xb3\xbc\xeb\x8c\x80\xed\x95\x99) 1\xed\x98\xb8\xea\xb4\x80 \xec\xa0\x95\xeb\xac\xb8 \xec\x95\x9e \xeb\xb2\xa4\xec\xb9\x98,"\xea\xb3\xb5\xeb\x8c\x80(\xea\xb3\xb5\xea\xb3\xbc\xeb\x8c\x80\xed\x95\x99) 1\xed\x98\xb8\xea\xb4\x80\xec\x9d\x98 \xec\xa0\x95\xeb\xac\xb8 \xec\x95\x9e \xec\x9e\x94\xeb\x94\x94\xeb\xb0\xad\xec\x97\x90\xeb\x8a\x94 \xeb\x82\x98\xeb\xac\xb4 \xeb\xb2\xa4\xec\xb9\x98\xea\xb0\x80 \xec\x84\xa4\xec\xb9\x98\xeb\x90\x98\xec\x96\xb4 \xec\x9e\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4.\n\xea\xb3\xb5\xeb\x8c\x80(\xea\xb3\xb5\xea\xb3\xbc\xeb\x8c\x80\xed\x95\x99) 1\xed\x98\xb8\xea\xb4\x80\xec\x97\x90\xec\x84\x9c \xec\x88\x98\xec\x97\x85\xec\x9d\x84 \xeb\x93\xa4\xec\x9d\x80 \xed\x95\x99\xec\x83\x9d\xeb\x93\xa4\xec\x9d\x80 \xea\xb3\xb5\xeb\x8c\x80(\xea\xb3\xb5\xea\xb3\xbc\xeb\x8c\x80\xed\x95\x99) 1\xed\x98\xb8\xea\xb4\x80 \x

In [ ]:
df = pd.read_csv("/content/place_info_corpus - query.csv")

In [ ]:
# 학습데이터 가공
new_rows = []

for index, row in df.iterrows():
  sentences = row['query'].split('\n')

  for sentence in sentences:
    new_row = row.copy()
    new_row['query'] = sentence
    new_rows.append(new_row)

new_df = pd.DataFrame(new_rows).drop('Unnamed: 5', axis=1)

new_df.reset_index(drop=False, inplace=True)
new_df.rename(columns = {"index": "docid"}, inplace=True)
new_df['qid'] = new_df.index
new_df.to_csv("new_query_data.csv", index=False)

In [ ]:
import csv

with open("new_query_data.csv", mode='r', encoding='utf-8') as file:
  reader = csv.DictReader(file)
  query_data = list(reader)

In [ ]:
# dataset split
from sklearn.model_selection import train_test_split

tmp_dataset, test_dataset = train_test_split(query_data, test_size = 0.2, random_state = 42, shuffle=True)

train_dataset, dev_dataset = train_test_split(tmp_dataset, test_size = 0.2, random_state=42, shuffle=True)

len(train_dataset)
len(dev_dataset)
len(test_dataset)

254

In [ ]:
# corpus 추출
keys_to_keep=['docid', 'corpus']

corpus_dataset = {one['docid']:one['corpus'] for one in query_data}
corpus_dataset.values()

dict_values(['공대(공과대학) 1호관의 정문 앞 잔디밭에는 나무 벤치가 설치되어 있습니다.\n공대(공과대학) 1호관에서 수업을 들은 학생들은 공대(공과대학) 1호관 정문 앞에 있는 벤치에 앉아서 휴식을 취할 수 있습니다.\n공대(공과대학) 1호관 정문 앞 나무 벤치는 주변에 나무가 많고 앞에 큰 돌이 있어요.\n공대(공과대학) 1호관 정문에 자리 잡은 벤치는 흡연구역과 가깝게 위치합니다.', '공대(공과대학) 편의점 근처에는 학생들이 즐겨 찾는 벤치가 있습니다.\n공대(공과대학) 편의점 앞의 벤치는 종종 학생들의 흡연 장소로 활용됩니다.\n공대(공과대학) 편의점 앞 벤치는 공대(공과대학) 1호관 뒤에 있습니다.\n공대(공과대학)의 편의점 옆에 위치한 벤치 근처에는 인쇄소가 있습니다.', '공대(공과대학) 2호관 옆에는 벤치들이 위치해 있습니다.\n공대(공과대학) 2호관 옆에 있는 벤치들 위에는 천막이 있어 비를 피할 수 있어요.\n공대(공과대학) 2호관 옆에는 벤치와 테이블 여러 개가 있습니다.\n공대(공과대학) 2호관 옆의 벤치에서 흡연하는 사람들이 많습니다.', '공대(공과대학) 1호관과 4호관 사이의 벤치는 두 건물 사이에 편의 시설을 제공합니다.\n공대(공과대학) 1호관과 4호관 사이의 벤치 옆에는 주차장이 있어요.\n공대(공과대학) 1호관과 4호관 사이에 위치한 벤치 앞에는 도서관으로 빠르게 갈 수 있는 계단이 있어요.\n공대(공과대학) 1호관과 공대(공과대학) 4호관 사이의 벤치는 학생들의 휴식 공간으로 인기있는 장소입니다.', '공대(공과대학) 2호관 정문 앞의 벤치 앞에는 주차장이 있습니다.\n공대(공과대학) 2호관 정문 앞 벤치는 공대(공과대학) 2호관 근처 나무들 사이에 있습니다.\n공대(공과대학) 2호관 정문에 위치한 벤치 근처에서 흡연하는 사람들이 많습니다.\n공대(공과대학) 2호관 정문 앞 벤치 뒤에는 학군단 건물이 있어요.', '공대(공과대학) 3호관과 9호관 사이의 벤치는 지붕이 있어요.\n공대(공과대학) 3호관과 9호관 

In [ ]:
dev_dataset

[{'docid': '47',
  'place_name': '계단9',
  'place_info': '공대(공과대학) 4호관과 공대(공과대학) 9호관 사이 계단',
  'corpus': '공대(공과대학) 4호관과 공대(공과대학) 9호관 사이에는 계단이 있습니다.\n공대(공과대학) 4호관과 옆 계단을 내려가면 건너편에 공대(공과대학) 8호관이 위치해 있습니다.\n공대(공과대학) 9호관 옆 계단 근처에는 지쿠터 주차장이 있습니다.\n공대(공과대학) 4호관과 공대(공과대학) 9호관 사이의 계단은 공대(공과대학) 8호관으로 향하는 주요 통로 역할을 합니다.',
  'coordinates (y, x)': '35.8475, 127.1333',
  'query': '공대(공과대학) 4호관과 공대(공과대학) 9호관 사이 계단에서 행사를 진행합니다.',
  'qid': '476'},
 {'docid': '61',
  'place_name': '지름길3',
  'place_info': '자연대(자연과학대학) 1호관에서 구정문 이어지는 지름길',
  'corpus': '자연대(자연과학대학) 1호관에서 구정문까지 가는 지름길이 있습니다. \n구정문에서 자연대(자연과학대학) 1호관으로 이어지는 지름길 근처에 주차장이 있습니다.\n자연대(자연과학대학) 1호관 앞에 있는 지름길을 통하면 구정문까지 바로 갈 수 있어요. \n구정문으로 가는 지름길은 자연대(자연과학대학) 1호관 옆에 있습니다.',
  'coordinates (y, x)': '35.845188, 127.127989',
  'query': '구정문과 자연대 1호관을 연결하는 이 지름길의 위치를 알려주세요.',
  'qid': '616'},
 {'docid': '100',
  'place_name': '흡연구역4',
  'place_info': '중앙도서관 앞 흡연구역',
  'corpus': '중앙도서관 정문 앞에 흡연구역에서 사람들이 담배를 핍니다.\n중앙도서관 앞 흡연구역은 벤치와 정자가 있습니다.\n중앙도서관 흡연구역 옆

In [ ]:
# dev 데이터셋 전처리
import random

dev_samples = {}

for one in dev_dataset:
  query, qid = one['query'], one['qid']
  positive = one['corpus']
  negative_data = [d for d in dev_dataset if d['docid'] != one['docid']]
  negatives = [d['corpus'] for d in negative_data]

  negative = random.sample(negatives, min(3, len(negatives)))

  dev_samples[qid] = {'query':query, 'positive':positive, 'negative': negative}
dev_samples

{'476': {'query': '공대(공과대학) 4호관과 공대(공과대학) 9호관 사이 계단에서 행사를 진행합니다.',
  'positive': '공대(공과대학) 4호관과 공대(공과대학) 9호관 사이에는 계단이 있습니다.\n공대(공과대학) 4호관과 옆 계단을 내려가면 건너편에 공대(공과대학) 8호관이 위치해 있습니다.\n공대(공과대학) 9호관 옆 계단 근처에는 지쿠터 주차장이 있습니다.\n공대(공과대학) 4호관과 공대(공과대학) 9호관 사이의 계단은 공대(공과대학) 8호관으로 향하는 주요 통로 역할을 합니다.',
  'negative': ['자연대(자연과학대학) 본관 옆에 위치한 정자는 도로변과 가까워요.\n자연대(자연과학대학) 본관 정자는 담배를 피우는 사람들이 자주 이용합니다.\n자연대(자연과학대학) 본관 정자 건너편에는 자연대(자연과학대학) 3호관이 있습니다.\n자연대(자연과학대학) 본관과 자연대(자연과학대학 3호관에서 수업을 학생들은 자연대(자연과학대학) 본관 옆 정자에서 잠시 휴식을 취하고는 합니다.',
   '자연대(자연과학대학) 본관 옆에 위치한 정자는 도로변과 가까워요.\n자연대(자연과학대학) 본관 정자는 담배를 피우는 사람들이 자주 이용합니다.\n자연대(자연과학대학) 본관 정자 건너편에는 자연대(자연과학대학) 3호관이 있습니다.\n자연대(자연과학대학) 본관과 자연대(자연과학대학 3호관에서 수업을 학생들은 자연대(자연과학대학) 본관 옆 정자에서 잠시 휴식을 취하고는 합니다.',
   '공대(공과대학) 편의점은 공대 1호관 뒤편에 있습니다. \n공대(공과대학) 건물 내에 위치한 편의점 옆에는 인쇄소가 있습니다. \n공대(공과대학) 편의점 앞에 벤치가 많습니다. \n공대(공과대학) 인근에 있는 편의점에서는 학생들이 모여 간단한 식사를 즐깁니다.']},
 '616': {'query': '구정문과 자연대 1호관을 연결하는 이 지름길의 위치를 알려주세요.',
  'positive': '자연대(자연과학대학) 1호관에서 구정문까지 가는 지름길이 있습니다. \

In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample
import os
from torch.utils.data import IterableDataset
from tqdm import tqdm
from torch.utils.data import Dataset

In [ ]:
from sentence_transformers.evaluation import RerankingEvaluator

In [ ]:
model_name = 'klue/bert-base'
use_pre_trained_model = False

train_batch_size = 16
max_seq_length = 256
num_epochs = 1
lr = 2e-5

In [ ]:
# 미리 학습되어있는 모델을 부르거나 or huggingface로부터 모델을 불러와 학습할 때
if use_pre_trained_model:
    print("use pretrained SBERT model")
    model = SentenceTransformer(model_name)
    model.max_seq_length = max_seq_length
else:
    print("Create new SBERT model")
    word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Create new SBERT model


In [ ]:
model_save_path = 'output/bi-encoder'

In [ ]:
import random
random.seed(1205)

In [ ]:
class MyCustomDataset(Dataset):
  def __init__(self, dataset):
    self.dataset = dataset

  def __getitem__(self, idx):
    data = self.dataset[idx]
    query = data['query']

    pos_passage = data['corpus']

    return InputExample(texts=[query, pos_passage])

  def __len__(self):
    return len(self.dataset)

In [ ]:
train_data = MyCustomDataset(query_data)
train_dataloader = DataLoader(train_data, shuffle=False, batch_size=train_batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [ ]:
# dev dataset에 대해 질의들의 MRR@10을 구한다.
evaluator = RerankingEvaluator(samples=dev_samples,
                   mrr_at_k=10, # similarity_fct=cos_sim,
                   batch_size=16,
                   show_progress_bar=True,
                   use_batched_encoding=True)

In [ ]:
import math

def callback(score, epoch, steps):
    print(f'epoch: {epoch}, steps: {steps}, MRR@10: {score}')
    if math.isnan(score):
        raise False


model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          evaluator=evaluator,
          evaluation_steps=10,
          warmup_steps=0,
          save_best_model=True,
          use_amp=True,
          checkpoint_path=model_save_path,
          checkpoint_save_steps=len(train_dataloader),
          optimizer_params = {'lr': lr},
          callback=callback,
          )

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/80 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/2331 [00:00<?, ?it/s]

epoch: 0, steps: 10, MRR@10: 0.9422566257179574


...

Batches:   0%|          | 0/2331 [00:00<?, ?it/s]

epoch: 0, steps: 80, MRR@10: 0.958517428476583


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/2331 [00:00<?, ?it/s]

epoch: 0, steps: -1, MRR@10: 0.958517428476583


In [ ]:
import joblib

joblib.dump(model, 'search_bi_encoder-gpu.pkl')

['search_bi_encoder-gpu.pkl']

In [ ]:
model.save('/output_pt')

In [ ]:
!zip -r model_building.zip '/content/output/bi-encoder'

  adding: content/output/bi-encoder/ (stored 0%)
  adding: content/output/bi-encoder/80/ (stored 0%)
  adding: content/output/bi-encoder/80/tokenizer.json (deflated 69%)
  adding: content/output/bi-encoder/80/sentence_bert_config.json (deflated 4%)
  adding: content/output/bi-encoder/80/special_tokens_map.json (deflated 42%)
  adding: content/output/bi-encoder/80/config.json (deflated 48%)
  adding: content/output/bi-encoder/80/1_Pooling/ (stored 0%)
  adding: content/output/bi-encoder/80/1_Pooling/config.json (deflated 47%)
  adding: content/output/bi-encoder/80/config_sentence_transformers.json (deflated 28%)
  adding: content/output/bi-encoder/80/model.safetensors (deflated 7%)
  adding: content/output/bi-encoder/80/README.md (deflated 58%)
  adding: content/output/bi-encoder/80/modules.json (deflated 53%)
  adding: content/output/bi-encoder/80/vocab.txt (deflated 49%)
  adding: content/output/bi-encoder/80/tokenizer_config.json (deflated 75%)


In [ ]:
files.download('model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sentence_transformers import util

In [ ]:
passages = list(corpus_dataset.values())

In [ ]:
corpus_embeddings = model.encode(passages, convert_to_tensor=True, show_progress_bar = True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def search_bi_encoder(query):
  question_embedding = model.encode(query, convert_to_tensor=True)
  question_embedding = question_embedding.cuda()
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=5)
  hits = hits[0]
  hits = sorted(hits, key=lambda x: x['score'], reverse=True)
  hits = [hit['corpus_id'] for hit in hits[:5]]
  return hits

In [ ]:
def compute_metrics(qids_to_relevant_documentids :dict, qids_to_ranked_candidate_documents :dict):
    """Compute MRR metric
    Args:
    p_qids_to_relevant_documentids (dict): dictionary of query-document mapping
        Dict as read in with load_reference or load_reference_from_stream
    p_qids_to_ranked_candidate_documents (dict): dictionary of query-document candidates
    Returns:
        dict: dictionary of metrics {'MRR': <MRR Score>}
    """
    MRR = 0
    all_scores = {}
    Hit_at_1 = 0
    Hit_at_10 = 0
    qids_with_relevant_documents = 0

    for qid in qids_to_ranked_candidate_documents:
        if qid in qids_to_relevant_documentids:
            target_pid = qids_to_relevant_documentids[qid]
            candidate_pid = qids_to_ranked_candidate_documents[qid]
            for i in range(0, len(candidate_pid[:10])):
                if candidate_pid[i] in target_pid:
                    if i == 0:
                        Hit_at_1 += 1
                    MRR += 1 / (i + 1)
                    Hit_at_10 += 1
                    break


    MRR = MRR / len(qids_to_relevant_documentids)
    Hit_at_1 = Hit_at_1 / len(qids_to_relevant_documentids)
    Hit_at_10 = Hit_at_10 / len(qids_to_relevant_documentids)
    all_scores['MRR @10'] = MRR
    all_scores['Hits @1'] = Hit_at_1
    all_scores['Hits @10'] = Hit_at_10
    return all_scores

In [ ]:
def docid2coord(docid):
  for one in query_data:
    if one['docid'] == docid:
      coord = one['coordinates (y, x)']
      break
  return coord

In [ ]:
docid2coord('115')

'35.8466, 127.1325'

In [ ]:
qrels, qid2cand = dict(), dict()
test_docids = list(corpus_dataset.keys())

for one in tqdm(test_dataset):
  qid, query = one['qid'], one['query']
  pos_ids = one['docid']
  cand = [test_docids[docid] for docid in search_bi_encoder(query)]
  qrels[qid] = pos_ids
  qid2cand[qid] = cand
  # 출력
  # print("\nquery: " + query)
  # for c in cand:
  #   print(corpus_dataset[c] + ' ' + docid2coord(c))
  # print('\n')

compute_metrics(qrels, qid2cand)

  4%|▍         | 11/254 [00:00<00:04, 48.64it/s]


query: 공대 5호관에서 강의가 있습니다.
공대(공과대학) 5호관 건물입니다.
공대(공과대학) 5호관은 소프트웨어공학과가 사용하는 건물입니다.
공대(공과대학) 5호관에서는 소프트웨어공학과가 주최하는 행사가 자주 열리는 장소입니다.
공대(공과대학) 5호관에는 소공 학생회실이 있습니다.
신소재공학부 전자재료공학 과사무실로 가려면 공대(공과대학) 5호관으로 가면 됩니다.
신소재공학부는 공대(공과대학) 5호관을 사용합니다.
정보소재 학생들은 공대(공과대학) 5호관을 자주 이용합니다.
전자재료 과사는 공대(공과대학) 5호관에 있습니다.
전자재료공학과 학생회실은 공대(공과대학) 5호관에 있습니다.
신소재공학부에서 열리는 행사는 주로 공대(공과대학) 5호관에서 열립니다.
전자계산소3은 공대(공과대학) 5호관 내에 위치하고 있습니다. 35.847535, 127.131514
자연대(자연과학대학) 본관에서 공대(공과대학) 5호관으로 이어지는 지름길을 올라오면 철봉이 있습니다.
공대(공과대학) 5호관 입구로 연결되는 다리 옆에는 철봉이 설치되어 있습니다.
공대(공과대학) 5호관 철봉 근처에는 큰 나무가 많습니다.
공대(공과대학) 5호관 철봉은 간단한 운동을 하는 데 이용됩니다. 35.8472, 127.1314
공대(공과대학) 5호관 옆에 지쿠터 주차장이 있습니다. 
공대 5호관 인근 지쿠터 주차장은 중앙도서관과 가깝습니다. 
공대(공과대학) 5호관 근처의 지쿠터 주차장에서 지쿠터가 필요한 사람들은 찾아서 사용할 수 있습니다. 
공대(공과대학) 5호관 지쿠터 주차장은 다른 공대들과 가깝지 않습니다. 35.847326, 127.131654
자연대(자연과학대학) 5호관 지쿠터 주차장은 자연대 5호관 계단 위에 있습니다. 
자연대 5호관 지쿠터 전용 주차장에서 출발하여 자연대(자연과학대학) 5호관 교실로 향하는 학생들을 자주 볼 수 있어요. 
자연대(자연과학대학) 5호관 지쿠터 주차장은 자연대(자연과학대학) 5호관 정문 바로 앞에 위치하고 있어요. 
자연대(자연과학대학) 5호관 지

  9%|▊         | 22/254 [00:00<00:04, 50.47it/s]


query: 공대 2호관 부속건물 옆 철봉에서 음식을 받을 예정입니다.
공대(공과대학) 2호관 뒤편에는 철봉이 있습니다.
공대(공과대학) 2호관 뒤에 설치된 철봉은 공대(공과대학) 2호관 뒤 벤치와 가깝습니다.
공대(공과대학) 2호관 부속건물 옆에는 철봉이 있습니다.
공대(공과대학) 2호관의 철봉은 학생들이 집중력을 향상시키기 위한 간단한 운동을 하는 데 이용됩니다. 35.8494, 127.1315
공대(공과대학) 2호관 뒤편 잔디밭에서 공대(공과대학) 5호관 방향으로 걸어가면 철봉이 있습니다.
자연대(자연과학대학) 본관에서 공대(공과대학) 2호관으로 이어지는 지름길을 올라오면 철봉이 설치되어 있습니다.
공대(공과대학) 2호관 철봉 근처에는 큰 나무가 많습니다.
공대(공과대학) 2호관 철봉은 간단한 운동을 하는 데 이용됩니다. 35.8472, 127.1314
자연대(자연과학대학) 2호관 앞에 부속 건물이 있습니다. 
자연대(자연과학대학) 2호관의 부속 건물은 사용하지 않는 낡은 건물입니다. 
자연대(자연과학대학) 2호관 부속 건물 앞에 있는 골목길에 꽃이 피어있습니다. 
자연대(자연과학대학) 2호관 부속 건물 옆에는 벤치가 있습니다. 35.844909, 127.129086
농대(농생대) 2호관 뒤쪽 잔디밭에는 벤치가 있습니다.
농대(농생대) 2호관 강의실이 있는 부속 건물 옆에는 벤치가 있습니다.
농대(농생대) 2호관 뒤쪽 벤치 앞에는 철봉이 있습니다.
농대(농생대) 2호관 뒤 벤치 주변에는 큰 나무들이 많습니다. 35.8494, 127.1315
공대(공과대학) 2호관 건물입니다.
유기소재섬유공학과(유소파, 유소섬), 토목환경자원에너지공학부(자원에너지공학), 일반화학준비실, 기초 실험실
공대(공과대학) 2호관은 유기소재섬유공학과 사용하는 건물입니다.
공대(공과대학) 2호관에는 유소파 학생회실이 있습니다.
유소섬 과사무실로 가려면 공대(공과대학) 2호관으로 가면 됩니다.
토목환경자원에너지공학부는 공대(공과대학) 2호관을 사용합니다.
자원에너지공학과 학생들은 공

 11%|█         | 28/254 [00:00<00:04, 48.48it/s]

...

 97%|█████████▋| 246/254 [00:05<00:00, 42.35it/s]


query: 농대 2호관 쓰레기 처리장에서 무언가를 시킬 수 있나요?
농대(농생대) 2호관 뒤편에는 쓰레기처리장이 있습니다.
농대(농생대) 2호관 쓰레기 처리장은 학생들이 일상적인 쓰레기와 재활용 가능한 폐기물을 적절하게 분리하여 처리할 수 있는 장소로 설정되어 있습니다.
 이 쓰레기 처리장은 농대(농생대) 2호관 주변의 폐기물을 체계적으로 관리하는 데 중요한 역할을 합니다.
 학생들은 농대(농생대) 2호관 쓰레기 처리장에서 일반 쓰레기를 쉽게 버리고, 재활용을 위한 분리수거를 할 수 있습니다. 35.849226, 127.131876
농대(농생대) 2호관 뒤쪽 잔디밭에는 벤치가 있습니다.
농대(농생대) 2호관 강의실이 있는 부속 건물 옆에는 벤치가 있습니다.
농대(농생대) 2호관 뒤쪽 벤치 앞에는 철봉이 있습니다.
농대(농생대) 2호관 뒤 벤치 주변에는 큰 나무들이 많습니다. 35.8494, 127.1315
농대(농생대) 2호관 앞에 지쿠터 주차장이 있습니다. 
농대(농생대) 2호관 지쿠터 주차장 근처에 벤치가 있습니다. 
농대(농생대) 2호관 근처에 위치한 지쿠터 주차장은 농대 2호관 정문 앞에 있습니다 . 
농대(농생대) 2호관 바로 옆에 조성된 지쿠터 주차장은 학생들에게 편리한 지쿠터 공간을 제공합니다. 35.848965, 127.131649
공대(공과대학) 2호관 뒤편에는 철봉이 있습니다.
공대(공과대학) 2호관 뒤에 설치된 철봉은 공대(공과대학) 2호관 뒤 벤치와 가깝습니다.
공대(공과대학) 2호관 부속건물 옆에는 철봉이 있습니다.
공대(공과대학) 2호관의 철봉은 학생들이 집중력을 향상시키기 위한 간단한 운동을 하는 데 이용됩니다. 35.8494, 127.1315
자연대(자연과학대학) 2호관 앞에 부속 건물이 있습니다. 
자연대(자연과학대학) 2호관의 부속 건물은 사용하지 않는 낡은 건물입니다. 
자연대(자연과학대학) 2호관 부속 건물 앞에 있는 골목길에 꽃이 피어있습니다. 
자연대(자연과학대학) 2호관 부속 건물 옆에는 벤치가 있습니다. 35.84

100%|██████████| 254/254 [00:05<00:00, 47.08it/s]


query: 자연대 5호관 앞에 위치한, 자주 사용되지 않는 벤치로 음식을 배달 부탁드립니다.
자연대(자연과학대학) 5호관 앞 인도에는 벤치가 설치되어 있습니다.
자연대(자연과학대학) 5호관 정문에서 인도로 나가는 길목에는 벤치가 있어요.
자연대(자연과학대학) 5호관 앞에 위치한 벤치는 학생들이 자주 사용하지 않습니다.
자연대(자연과학대학) 5호관 앞 벤치는 학생들이 많이 지나는 길에 있습니다. 35.845819, 127.130906
자연대(자연과학대학) 5호관 지쿠터 주차장은 자연대 5호관 계단 위에 있습니다. 
자연대 5호관 지쿠터 전용 주차장에서 출발하여 자연대(자연과학대학) 5호관 교실로 향하는 학생들을 자주 볼 수 있어요. 
자연대(자연과학대학) 5호관 지쿠터 주차장은 자연대(자연과학대학) 5호관 정문 바로 앞에 위치하고 있어요. 
자연대(자연과학대학) 5호관 지쿠터 주차장은 rotc 건물 옆에 위치하고 있습니다. 35.845951, 127.130883
자연대(자연과학대학) 5호관 주차장 앞에는 계단이 있습니다.
자연대(자연과학대학) 5호관 주차장 앞 계단은 자연대(자연과학대학) 5호관 입구와 이어집니다.
자연대(자연과학대학) 5호관 앞 계단을 올라가면 지쿠터 주차장이 있습니다.
자연대(자연과학대학) 5호관 인근의 계단은 학생들이 편리하게 이동할 수 있는 경로입니다. 35.845921, 127.130821
자연대(자연과학대학) 본관에서 공대(공과대학) 5호관으로 이어지는 지름길을 올라오면 철봉이 있습니다.
공대(공과대학) 5호관 입구로 연결되는 다리 옆에는 철봉이 설치되어 있습니다.
공대(공과대학) 5호관 철봉 근처에는 큰 나무가 많습니다.
공대(공과대학) 5호관 철봉은 간단한 운동을 하는 데 이용됩니다. 35.8472, 127.1314
공대(공과대학) 4호관과 공대(공과대학) 5호관 사이에는 벤치가 있습니다.
중앙도서관에서 공대(공과대학) 2호관으로 올라오는 계단 앞에는 벤치가 있습니다.
공대(공과대학) 4호관 벤치 앞에는 지쿠터 주차장이 있습니다

{'MRR @10': 0.8824803149606301,
 'Hits @1': 0.8307086614173228,
 'Hits @10': 0.9488188976377953}

In [ ]:
search_bi_encoder("도시공학과 사무실은 어디 있어?")

[108, 121, 31, 123, 125]

In [ ]:
passages[108]

'목재응용과학과 학생회실은 농대 안에 있습니다. \n목재응용과학과의 학생회실은 농대 4호관 뒤에 있습니다. \n목재응용과학과 학생들의 학생회실로, 작은 컨테이너 같은 건물입니다. \n목재응용과학과 학생회실은 목재응용학과 강의실과 다른 건물에 있습니다.'

In [ ]:
corpus_dataset['108']

'목재응용과학과 학생회실은 농대 안에 있습니다. \n목재응용과학과의 학생회실은 농대 4호관 뒤에 있습니다. \n목재응용과학과 학생들의 학생회실로, 작은 컨테이너 같은 건물입니다. \n목재응용과학과 학생회실은 목재응용학과 강의실과 다른 건물에 있습니다.'

In [ ]:
for one in tqdm(test_dataset):
  qid = one['qid']
  print("query: " + one['query'])
  print("search_passage : " + corpus_dataset[qrels[qid]])
  print("positive_passage : " + one['corpus'])

100%|██████████| 233/233 [00:00<00:00, 8429.42it/s]

query: 농대 본관 정문 정자, 학술림과 가까운 곳으로 음식을 배달 부탁드립니다.
search_passage : 농대(농생대) 본관 정문 앞 잔디밭에는 정자가 있습니다.
농대(농생대) 본관을 향해 서 있을 때, 오른쪽 주차장 뒤편에 정자가 있습니다.
농대(농생대) 본관 정문 정자는 학술림과 가까워요.
농대(농생대) 본관 정문 정자 뒤편에는 공대(공과대학) 8호관이 위치해 있습니다.
positive_passage : 농대(농생대) 본관 정문 앞 잔디밭에는 정자가 있습니다.
농대(농생대) 본관을 향해 서 있을 때, 오른쪽 주차장 뒤편에 정자가 있습니다.
농대(농생대) 본관 정문 정자는 학술림과 가까워요.
농대(농생대) 본관 정문 정자 뒤편에는 공대(공과대학) 8호관이 위치해 있습니다.
query: 공대 6호관과 후생관 사이 벤치에서 손님을 맞이하려고 합니다 이곳으로 와 주실 수 있나요?
search_passage : 공대(공과대학) 6호관과 후생관 사이에는 벤치가 위치해 있습니다.
보건진료소 앞에서 후생관 앞을 지나는 길에는 벤치가 있어요.
공대(공과대학) 6호관과 후생관 사이에 위치한 벤치는 후생관에서 식사를 하기 위해 기다리는 사람들에게 적합합니다.
후생관에서 나온 학생들은 후생관과 공대(공과대학) 6호관 사이에 위치한 벤치에서 잠시 대화를 나누기도 합니다.
positive_passage : 공대(공과대학) 6호관과 후생관 사이에는 벤치가 위치해 있습니다.
보건진료소 앞에서 후생관 앞을 지나는 길에는 벤치가 있어요.
공대(공과대학) 6호관과 후생관 사이에 위치한 벤치는 후생관에서 식사를 하기 위해 기다리는 사람들에게 적합합니다.
후생관에서 나온 학생들은 후생관과 공대(공과대학) 6호관 사이에 위치한 벤치에서 잠시 대화를 나누기도 합니다.
query: 중앙도서관에 있는 공터은 어디를 말하는건가요?
search_passage : 중앙도서관 앞에 공터가 있습니다. 
중앙도서관 공터는 학생들이 대규모 행사를 열 때 자주 사용하는 공간입니다. 
중앙도서관 인근의 넓